In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
from datetime import datetime

In [5]:
driver = webdriver.Chrome('/Users/parkkyuseon/Downloads/chromedriver')
driver.implicitly_wait(3)

In [6]:
driver.get('https://www.baseball-reference.com/leagues/MLB/2020.shtml')
driver

<selenium.webdriver.chrome.webdriver.WebDriver (session="5371e6dd9486bd3edd3c2f6109880069")>

In [21]:
col_team_off = [['Yr', 'Team', 'R/G', 'G', 'PA', 'AB', 'R', 'H', '2B', '3B', 'HR', 
                 'RBI', 'SB', 'CS', 'BB', 'SO', 'BA', 'OBP', 'SLG', 'OPS', 'OPS+', 'TB',
                 'GDP', 'HBP', 'SH', 'SF', 'IBB', 'LOB']]
col_team_def = [['Yr', 'Team', 'RA/G', 'W', 'L', 'WP', 'ERA', 'G', 'GS', 'GF', 'CG', 'tSho', 'cSho', 'SV',
                 'IP', 'H', 'R', 'ER', 'HR', 'BB', 'IBB', 'SO', 'HBP', 'BK', 'WP', 'BF', 'ERA+', 'FIP',
                 'WHIP', 'H9', 'HR9', 'BB9', 'SO9', 'SO/W', 'LOB']]

team_off = pd.DataFrame(columns = col_team_off)
team_def = pd.DataFrame(columns = col_team_def)

In [22]:
## Edit Year to Get Previous Data
year = range(2020, 2000, -1)
for yr in year:
    url = 'https://www.baseball-reference.com/leagues/MLB/'+str(yr)+'.shtml'
    driver.get(url)

    driver.implicitly_wait(15)
    table = driver.find_element_by_xpath('//*[@id="teams_standard_batting"]/tbody').get_attribute("innerHTML") 
    soup = BeautifulSoup(table, 'html.parser') 

    for i in range(0,len(soup.findAll('tr'))-1):
        team_name = soup.findAll('tr')[i].get_text()[0:3]
        data = soup.findAll('tr')[i]
        data = data.findAll('td')[2::]
        for j in range(0, len(data)):
            data[j] = data[j].get_text()
        data = [yr, team_name] + data
        data = pd.DataFrame(data).T
        data.columns = team_off.columns
        team_off = pd.concat([team_off, data])
        
    table = driver.find_element_by_xpath('//*[@id="teams_standard_pitching"]/tbody').get_attribute("innerHTML") 
    soup = BeautifulSoup(table, 'html.parser') 
    
    for i in range(0,len(soup.findAll('tr'))-1):
        team_name = soup.findAll('tr')[i].get_text()[0:3]
        data = soup.findAll('tr')[i]
        data = data.findAll('td')[2::]
        for j in range(0, len(data)):
            data[j] = data[j].get_text()
        data = [yr, team_name] + data
        data = pd.DataFrame(data).T
        data.columns = team_def.columns
        team_def = pd.concat([team_def, data])

In [23]:
team_off

Yr Team   R/G    G    PA    AB    R     H   2B  3B  ...   SLG   OPS  \
0   2020  ARI  4.48   60  2238  1997  269   482  101  12  ...  .391  .704   
0   2020  ATL  5.80   60  2344  2074  348   556  130   3  ...  .483  .832   
0   2020  BAL  4.57   60  2242  2026  274   523  102   7  ...  .429  .750   
0   2020  BOS  4.87   60  2304  2083  292   552  118   7  ...  .445  .775   
0   2020  CHC  4.42   60  2214  1918  265   422   82   8  ...  .387  .705   
..   ...  ...   ...  ...   ...   ...  ...   ...  ...  ..  ...   ...   ...   
0   2001  SFG  4.93  162  6408  5612  799  1493  304  40  ...  .460  .802   
0   2001  STL  5.02  162  6177  5450  814  1469  274  32  ...  .441  .780   
0   2001  TBD  4.15  162  6104  5524  672  1426  311  21  ...  .388  .707   
0   2001  TEX  5.49  162  6388  5685  890  1566  326  23  ...  .471  .815   
0   2001  TOR  4.73  162  6284  5663  767  1489  287  36  ...  .430  .755   

   OPS+    TB  GDP HBP  SH  SF IBB   LOB  
0    87   781   37  36   1  23   5   399  
0   116  1001   39  23   1   7  13   437  
0   104   870   32  27  15  10   5   399  
0   107   927   51  21   4   9   8   422  
0    90   742   42  52   1  13   9   406  
..  ...   ...  ...  ..  ..  ..  ..   ...  
0   112  2582  108  50  67  54  79  1242  
0   100  2404  125  65  83  50  51  1095  
0    88  2142  130  54  45  25  24  1104  
0   109  2676  131  75  25  55  27  1161  
0    96  2433  111  74  34  43  47  1124  

[600 rows x 28 columns]

In [24]:
team_def

Yr Team  RA/G   W    L    WP   ERA    G   GS   GF  ...    BF ERA+   FIP  \
0   2020  ARI  4.92  25   35  .417  4.84   60   60   60  ...  2279   96  4.99   
0   2020  ATL  4.80  35   25  .583  4.41   60   60   60  ...  2276  108  4.42   
0   2020  BAL  4.90  25   35  .417  4.51   60   60   60  ...  2246  102  4.60   
0   2020  BOS  5.85  24   36  .400  5.58   60   60   60  ...  2389   85  5.19   
0   2020  CHC  4.00  34   26  .567  3.99   60   60   58  ...  2153  113  4.18   
..   ...  ...   ...  ..  ...   ...   ...  ...  ...  ...  ...   ...  ...   ...   
0   2001  SFG  4.62  90   72  .556  4.18  162  162  159  ...  6294   95  4.12   
0   2001  STL  4.22  93   69  .574  3.93  162  162  154  ...  6121  110  4.57   
0   2001  TBD  5.48  62  100  .383  4.94  162  162  161  ...  6288   90  4.85   
0   2001  TEX  5.98  73   89  .451  5.71  162  162  158  ...  6469   82  5.11   
0   2001  TOR  4.65  80   82  .494  4.28  162  162  155  ...  6305  107  4.25   

     WHIP    H9  HR9  BB9  SO9  SO/W   LOB  
0   1.430   8.8  1.6  4.1  9.1  2.23   429  
0   1.362   8.5  1.2  3.8  8.7  2.30   415  
0   1.313   8.5  1.4  3.3  8.5  2.54   396  
0   1.601  10.1  1.7  4.3  9.2  2.13   466  
0   1.221   7.8  1.3  3.2  9.1  2.87   358  
..    ...   ...  ...  ...  ...   ...   ...  
0   1.378   8.8  0.9  3.6  6.6  1.87  1156  
0   1.334   8.7  1.2  3.3  6.8  2.06  1131  
0   1.462   9.6  1.3  3.6  6.5  1.81  1130  
0   1.575  10.4  1.4  3.7  6.0  1.60  1186  
0   1.397   9.6  1.0  3.0  6.4  2.12  1164  

[600 rows x 35 columns]

In [66]:
col_team_pitWAR = [['yr', 'team', 'PitWAR']]
col_team_offWAR = [['yr', 'team', 'offWAR']]

team_pitWAR = pd.DataFrame(columns = col_team_pitWAR)
team_offWAR = pd.DataFrame(columns = col_team_offWAR)

In [67]:
## Edit Year to Get Previous Data
year = range(2020, 2010, -1)
for yr in year:
    url = 'https://www.baseball-reference.com/leagues/team_compare.cgi?request=1&year='+str(yr)+'lg=MLB&stat=WAR'
    driver.get(url)

    driver.implicitly_wait(5)
    table = driver.find_element_by_xpath('//*[@id="team_output"]/tbody').get_attribute("innerHTML") 
    soup = BeautifulSoup(table, 'html.parser') 

    for i in range(0,len(soup.findAll('tr'))):
        data = soup.findAll('tr')[i]
        data = data.findAll('td')[1].get_text()
        data = [data[0:3]] + [data[3:]]
        data = [yr] + data
        data = pd.DataFrame(data).T
        data.columns = team_pitWAR.columns
        team_pitWAR = pd.concat([team_pitWAR, data])
        
    table = driver.find_element_by_xpath('//*[@id="team_output"]/tbody').get_attribute("innerHTML") 
    soup = BeautifulSoup(table, 'html.parser') 

    for i in range(0,len(soup.findAll('tr'))):
        data = soup.findAll('tr')[i]
        data = data.findAll('td')[4].get_text()
        data = [data[0:3]] + [data[3:]]
        data = [yr] + data
        data = pd.DataFrame(data).T
        data.columns = team_offWAR.columns
        team_offWAR = pd.concat([team_offWAR, data])

In [68]:
team_pitWAR

yr team PitWAR
0   2020  CLE    9.8
0   2020  BAL    8.6
0   2020  ATL    8.3
0   2020  WSN    8.3
0   2020  TBR    7.2
..   ...  ...    ...
0   2011  NYM    6.7
0   2011  CHC    6.3
0   2011  CIN    5.5
0   2011  BAL    4.9
0   2011  HOU   -2.0

[300 rows x 3 columns]

In [69]:
team_offWAR

yr team offWAR
0   2020  LAD   12.9
0   2020  SDP   10.1
0   2020  NYY    9.9
0   2020  CHW    9.7
0   2020  TBR    9.6
..   ...  ...    ...
0   2011  CHW   10.5
0   2011  OAK   10.4
0   2011  PIT   10.0
0   2011  FLA    9.4
0   2011  MIN    3.8

[300 rows x 3 columns]

In [115]:
col_team_info = [['yr', 'Team', 'League', 'G', 'W', 'L', 'WPCT', 'R', 'RA']]

team_info = pd.DataFrame(columns = col_team_info)

In [116]:
url = 'https://www.baseball-reference.com/leagues/MLB/2020-standings.shtml'
driver.get(url)

driver.implicitly_wait(5)
table = driver.find_element_by_xpath('//*[@id="expanded_standings_overall"]/tbody').get_attribute("innerHTML") 
soup = BeautifulSoup(table, 'html.parser') 

for i in range(0,len(soup.findAll('tr'))-1):
    data = soup.findAll('tr')[i]
    data = data.findAll('td')[0:9]
    del data[6]
    for j in range(0, len(data)):
        data[j] = data[j].get_text()
    data = [2020] + data
    data = pd.DataFrame(data).T
    data.columns = team_info.columns
    team_info = pd.concat([team_info, data])

In [154]:
for i in range(0, len(team_info[['Team']])):
    if len(team_info.iloc[i, 1]) == 5:
        team_info.iloc[i, 1] = team_info.iloc[i, 1][2:5]

team_info

yr Team League   G   W   L  WPCT    R   RA
0  2020  LAD     NL  60  43  17  .717  5.8  3.6
0  2020  TBR     AL  60  40  20  .667  4.8  3.8
0  2020  SDP     NL  60  37  23  .617  5.4  4.0
0  2020  MIN     AL  60  36  24  .600  4.5  3.6
0  2020  OAK     AL  60  36  24  .600  4.6  3.9
0  2020  ATL     NL  60  35  25  .583  5.8  4.8
0  2020  CHW     AL  60  35  25  .583  5.1  4.1
0  2020  CLE     AL  60  35  25  .583  4.1  3.5
0  2020  CHC     NL  60  34  26  .567  4.4  4.0
0  2020  NYY     AL  60  33  27  .550  5.3  4.5
0  2020  TOR     AL  60  32  28  .533  5.0  5.2
0  2020  STL     NL  58  30  28  .517  4.1  3.9
0  2020  MIA     NL  60  31  29  .517  4.4  5.1
0  2020  CIN     NL  60  31  29  .517  4.1  4.1
0  2020  HOU     AL  60  29  31  .483  4.7  4.6
0  2020  SFG     NL  60  29  31  .483  5.0  5.0
0  2020  MIL     NL  60  29  31  .483  4.1  4.4
0  2020  PHI     NL  60  28  32  .467  5.1  5.2
0  2020  SEA     AL  60  27  33  .450  4.2  5.1
0  2020  NYM     NL  60  26  34  .433  4.8  5.1
0  2020  COL     NL  60  26  34  .433  4.6  5.9
0  2020  KCR     AL  60  26  34  .433  4.1  4.5
0  2020  LAA     AL  60  26  34  .433  4.9  5.4
0  2020  WSN     NL  60  26  34  .433  4.9  5.0
0  2020  BAL     AL  60  25  35  .417  4.6  4.9
0  2020  ARI     NL  60  25  35  .417  4.5  4.9
0  2020  BOS     AL  60  24  36  .400  4.9  5.9
0  2020  DET     AL  58  23  35  .397  4.3  5.5
0  2020  TEX     AL  60  22  38  .367  3.7  5.2
0  2020  PIT     NL  60  19  41  .317  3.7  5.0

In [155]:
## Edit Year to Get Previous Data
## Do Not Put 2020 Here

year = range(2019, 2010, -1)
for yr in year:
    url = 'https://www.baseball-reference.com/leagues/MLB/'+str(yr)+'-standings.shtml'
    driver.get(url)

    driver.implicitly_wait(5)
    table = driver.find_element_by_xpath('//*[@id="expanded_standings_overall"]/tbody').get_attribute("innerHTML") 
    soup = BeautifulSoup(table, 'html.parser') 

    for i in range(0,len(soup.findAll('tr'))-1):
        data = soup.findAll('tr')[i]
        data = data.findAll('td')[0:8]
        for j in range(0, len(data)):
            data[j] = data[j].get_text()
        data = [yr] + data
        data = pd.DataFrame(data).T
        data.columns = team_info.columns
        team_info = pd.concat([team_info, data])
        
team_info

yr Team League    G   W    L  WPCT    R   RA
0   2020  LAD     NL   60  43   17  .717  5.8  3.6
0   2020  TBR     AL   60  40   20  .667  4.8  3.8
0   2020  SDP     NL   60  37   23  .617  5.4  4.0
0   2020  MIN     AL   60  36   24  .600  4.5  3.6
0   2020  OAK     AL   60  36   24  .600  4.6  3.9
..   ...  ...    ...  ...  ..  ...   ...  ...  ...
0   2011  SDP     NL  162  71   91  .438  3.7  3.8
0   2011  BAL     AL  162  69   93  .426  4.4  5.3
0   2011  SEA     AL  162  67   95  .414  3.4  4.2
0   2011  MIN     AL  162  63   99  .389  3.8  5.0
0   2011  HOU     NL  162  56  106  .346  3.8  4.9

[300 rows x 9 columns]